## Build Network
Notebook to establish networks for evaluation with "Run Network.ipynb". Constructs a model "V1" of L4 Exc, PV, SST, And VIP neurons using simple LIF dynamics. Also creates a virtual network "LGN" to provide input to this internal network. 

### Create network nodes
Establish nodes for each cell type, arranged in 2D space (not worrying about spatial overlap of cell types). Numbers for each cell type are calculated within "Cell Type Fractions.ipynb"

In [1]:
from bmtk.builder.networks import NetworkBuilder
from bmtk.builder.auxi.node_params import positions_columinar
from bmtk.builder.auxi.node_params import positions_cuboid
from time import gmtime, strftime

# Cell counts
nExc = 8517
nPV = 800
nSST = 459
nVIP = 224

# Create network "V1" which contains nodes for excitatory, PV, SST, and VIP cells arranged in 2D space
net = NetworkBuilder("V1")
net.add_nodes(N=nExc, 
              positions=positions_cuboid(N=nExc,center=[0, 0, 0], height = 0, xside_length=100, \
                                         yside_length=100, min_dist=1),
              pop_name='Exc', location='VisL4', ei='e',  # optional parameters
              model_type='point_process',  # Tells the simulator to use point-based neurons
              model_template='nest:iaf_psc_alpha',  # tells the simulator to use NEST iaf_psc_alpha models
              dynamics_params='IntFire1_exc_point.json'  # File containing iaf_psc_alpha mdoel parameters
             )

net.add_nodes(N=nPV, pop_name='PV', location='VisL4', ei='i',
              positions=positions_cuboid(N=nPV,center=[0, 0, 0], height = 0, xside_length=100, \
                                         yside_length=100, min_dist=1),
              model_type='point_process',
              model_template='nest:iaf_psc_alpha',
              dynamics_params='IntFire1_inh_point.json')

net.add_nodes(N=nSST, pop_name='SST', location='VisL4', ei='i',
              positions=positions_cuboid(N=nSST,center=[0, 0, 0], height = 0, xside_length=100, \
                                         yside_length=100, min_dist=1),
              model_type='point_process',
              model_template='nest:iaf_psc_alpha',
              dynamics_params='IntFire1_inh_point.json')

net.add_nodes(N=nVIP, pop_name='VIP', location='VisL4', ei='i',
              positions=positions_cuboid(N=nVIP,center=[0, 0, 0], height = 0, xside_length=100, \
                                         yside_length=100, min_dist=1),
              model_type='point_process',
              model_template='nest:iaf_psc_alpha',
              dynamics_params='IntFire1_inh_point.json')

print('Nodes established.')
print(strftime("%Y-%m-%d %H:%M:%S", gmtime())) #timestamp

Nodes established.
2022-01-20 08:40:32


### Create network edges
Establish synaptic connections between nodes of network, using a connection rule that randomly assigns one or zero synapses based on the probability of given cell-type pairwise probability. Pairwise probability come from Allen connectivity datasets, and are located in "Cell Type Fractions.ipynb"

In [2]:
import numpy as np

# Function to assign a synapse for given synaptic probabilities
def calc_synapses(sources, target, probability):
    syns = int(np.random.rand(1) < probability)
    return syns

## E-to-E connections
net.add_edges(source={'pop_name': 'Exc'}, target={'pop_name': 'Exc'},
              connection_rule=calc_synapses,
              connection_params={'probability': .103},
              syn_weight=3.0,
              delay=2.0,
              dynamics_params='ExcToExc.json',
              model_template='static_synapse')

### Generating I-to-I connections

## PV I-to-I
net.add_edges(source={'pop_name': 'PV'}, target={'pop_name': 'PV'},
              connection_rule=calc_synapses,
              connection_params={'probability': .472},
              syn_weight=-3.0,
              delay=2.0,
              dynamics_params='InhToInh.json',
              model_template='static_synapse')

net.add_edges(source={'pop_name': 'PV'}, target={'pop_name': 'SST'},
              connection_rule=calc_synapses,
              connection_params={'probability': .0},
              syn_weight=-3.0,
              delay=2.0,
              dynamics_params='InhToInh.json',
              model_template='static_synapse')

net.add_edges(source={'pop_name': 'PV'}, target={'pop_name': 'VIP'},
              connection_rule=calc_synapses,
              connection_params={'probability': .088},
              syn_weight=-3.0,
              delay=2.0,
              dynamics_params='InhToInh.json',
              model_template='static_synapse')

## SST I-to-I
net.add_edges(source={'pop_name': 'SST'}, target={'pop_name': 'PV'},
              connection_rule=calc_synapses,
              connection_params={'probability': .0},
              syn_weight=-3.0,
              delay=2.0,
              dynamics_params='InhToInh.json',
              model_template='static_synapse')

net.add_edges(source={'pop_name': 'SST'}, target={'pop_name': 'SST'},
              connection_rule=calc_synapses,
              connection_params={'probability': .0},
              syn_weight=-3.0,
              delay=2.0,
              dynamics_params='InhToInh.json',
              model_template='static_synapse')

net.add_edges(source={'pop_name': 'SST'}, target={'pop_name': 'VIP'},
              connection_rule=calc_synapses,
              connection_params={'probability': .286},
              syn_weight=-3.0,
              delay=2.0,
              dynamics_params='InhToInh.json',
              model_template='static_synapse')

## VIP I-to-I
net.add_edges(source={'pop_name': 'VIP'}, target={'pop_name': 'PV'},
              connection_rule=calc_synapses,
              connection_params={'probability': .022},
              syn_weight=-3.0,
              delay=2.0,
              dynamics_params='InhToInh.json',
              model_template='static_synapse')

net.add_edges(source={'pop_name': 'VIP'}, target={'pop_name': 'SST'},
              connection_rule=calc_synapses,
              connection_params={'probability': .172},
              syn_weight=-3.0,
              delay=2.0,
              dynamics_params='InhToInh.json',
              model_template='static_synapse')

net.add_edges(source={'pop_name': 'VIP'}, target={'pop_name': 'VIP'},
              connection_rule=calc_synapses,
              connection_params={'probability': .023},
              syn_weight=-3.0,
              delay=2.0,
              dynamics_params='InhToInh.json',
              model_template='static_synapse')


### Generating I-to-E connections
net.add_edges(source={'pop_name': 'PV'}, target={'pop_name': 'Exc'},
              connection_rule=calc_synapses,
              connection_params={'probability': .200},
              syn_weight=-3.0,
              delay=2.0,
              dynamics_params='InhToExc.json',
              model_template='static_synapse')

net.add_edges(source={'pop_name': 'SST'}, target={'pop_name': 'Exc'},
              connection_rule=calc_synapses,
              connection_params={'probability': .222},
              syn_weight=-3.0,
              delay=2.0,
              dynamics_params='InhToExc.json',
              model_template='static_synapse')

net.add_edges(source={'pop_name': 'VIP'}, target={'pop_name': 'Exc'},
              connection_rule=calc_synapses,
              connection_params={'probability': .0},
              syn_weight=-3.0,
              delay=2.0,
              dynamics_params='InhToExc.json',
              model_template='static_synapse')

### Generating E-to-I connections
net.add_edges(source={'pop_name': 'Exc'}, target={'pop_name': 'PV'},
              connection_rule=calc_synapses,
              connection_params={'probability': .114},
              syn_weight=3.0,
              delay=2.0,
              dynamics_params='ExcToInh.json',
              model_template='static_synapse')

net.add_edges(source={'pop_name': 'Exc'}, target={'pop_name': 'SST'},
              connection_rule=calc_synapses,
              connection_params={'probability': .036},
              syn_weight=3.0,
              delay=2.0,
              dynamics_params='ExcToInh.json',
              model_template='static_synapse')

net.add_edges(source={'pop_name': 'Exc'}, target={'pop_name': 'VIP'},
              connection_rule=calc_synapses,
              connection_params={'probability': .038},
              syn_weight=3.0,
              delay=2.0,
              dynamics_params='ExcToInh.json',
              model_template='static_synapse')



print('Edges established.')
print(strftime("%Y-%m-%d %H:%M:%S", gmtime())) #timestamp

Edges established.
2022-01-20 08:40:34


In [3]:
# Build and save network "V1"
net.build()
net.save_nodes(output_dir='simulation/network')
net.save_edges(output_dir='simulation/network')

### Building external network

Next we want to create an external network of "virtual cells" with spike-trains that will synapse onto our V1 cells and drive activity. We will call this external network "LGN" and contains 500 excitatory cells.

In [3]:
# Build LGN with positions that overlap V1 network to establish retinotopy
nLGN = 1000

lgn = NetworkBuilder('LGN')
lgn.add_nodes(N=nLGN, pop_name='tON', potential='exc', model_type='virtual', \
              positions=positions_cuboid(N=nLGN,center=[0, 0, 0], height = 0, xside_length=100, yside_length=100, min_dist=1),)


We will use a special function for setting the number of synapses between the LGN --> V1 cells. The select_source_cells function will be called during the build process.

In [4]:
import numpy as np

# Function to retinotopically assign LGN synaptic targets
def select_sources_retinotopically(source,target,d_max):
    # Determine distance between source and target
    d = np.linalg.norm(np.array(source['positions']) - np.array(target['positions']))

    # Calculate fraction of max allowable distance
    d_ratio = d / d_max
    
    # Calculate likelihood of a synapse
    syns = int(np.random.rand(1) > d_ratio)*5
    return syns

lgn.add_edges(source=lgn.nodes(), target=net.nodes(pop_name='Exc'),
              connection_rule= select_sources_retinotopically,
              connection_params={'d_max': 10},
              syn_weight=15.0,
              delay=2.0,
              dynamics_params='ExcToExc.json',
              model_template='static_synapse')

lgn.add_edges(source=lgn.nodes(), target=net.nodes(pop_name='PV'),
              connection_rule= select_sources_retinotopically,
              connection_params={'d_max': 10},
              syn_weight=5.0,
              delay=2.0,
              dynamics_params='ExcToInh.json',
              model_template='static_synapse')

lgn.add_edges(source=lgn.nodes(),  target=net.nodes(pop_name='SST'),
              connection_rule= select_sources_retinotopically,
              connection_params={'d_max': 10},
              syn_weight= 5.0,
              delay=2.0,
              dynamics_params='ExcToInh.json',
              model_template='static_synapse')

lgn.add_edges(source=lgn.nodes(),  target=net.nodes(pop_name='VIP'),
              connection_rule= select_sources_retinotopically,
              connection_params={'d_max': 10},
              syn_weight= 5.0,
              delay=2.0,
              dynamics_params='ExcToInh.json',
              model_template='static_synapse')

print('LGN edges established.')
print(strftime("%Y-%m-%d %H:%M:%S", gmtime())) #timestamp

LGN edges established.
2022-01-20 08:40:39


In [10]:
# Set up LGN firing
from bmtk.utils.reports.spike_trains import PoissonSpikeGenerator

psg = PoissonSpikeGenerator(population='LGN')
psg.add(node_ids=range(nLGN),  # Have nodes match LGN
        firing_rate=15.0,    # 15 Hz, verify later
        times=(0.25, 1.0))    # Firing starts at 0 s up to 3 s
psg.add(node_ids=range(nLGN),  # Have nodes match LGN
        firing_rate=22.0,    # 15 Hz, verify later
        times=(1.01, 1.75))    # Firing starts at 0 s up to 3 s
psg.add(node_ids=range(nLGN),  # Have nodes match LGN
        firing_rate=5.0,    # 15 Hz, verify later
        times=(1.76, 2.25))    # Firing starts at 0 s up to 3 s
psg.add(node_ids=range(nLGN),  # Have nodes match LGN
        firing_rate=15.0,    # 15 Hz, verify later
        times=(2.26, 3))    # Firing starts at 0 s up to 3 s
psg.to_sonata('simulation/inputs/lgn_spikes.h5')

# Validate total spike number: Spikes = neurons * rate * time
print('Number of spikes: {}'.format(psg.n_spikes()))


Number of spikes: 41234


In [6]:
lgn.build()
lgn.save_nodes(output_dir='simulation/network')
lgn.save_edges(output_dir='simulation/network')

## Setting up PointNet Environment


In [8]:
from bmtk.utils.sim_setup import build_env_pointnet

build_env_pointnet(base_dir='simulation',      
                   network_dir='simulation/network',
                   tstop=3000.0, 
                   dt=0.01,
                   spikes_inputs=[('LGN','simulation/inputs/lgn_spikes.h5')],
                   include_examples=True,         # Copies components files
                  )


print('Network built.')
print(strftime("%Y-%m-%d %H:%M:%S", gmtime())) #timestamp

Network built.
2022-01-20 08:33:35


The network files are written to **circuit_config.json** and the simulation parameters are set in **simulation_config**. The simulation time is set to run for 3000.0 ms (tstop). We also specify a membrane-report to record V_m property of 4 cells (gids 0, 80, 100, 300 - one from each cell-type). In general, all the parameters needed to setup and start a simulation are found in the config files, and adjusting network/simulation conditions can be done by editing these json files in a text editor.

#### Additional validation scripts

In [5]:
# Script to check various node values are populating as expected
v1_nodes = list(net.nodes())
n0 = v1_nodes[0]
n1 = v1_nodes[1]
n2 = v1_nodes[2]
n3 = v1_nodes[3]
n4 = v1_nodes[4]
print('cell 0: pop_name: {}, positions: {}'.format(n0['pop_name'], n0['positions']))
print('cell 1: pop_name: {}, positions: {}'.format(n1['pop_name'], n1['positions']))
print('cell 2: pop_name: {}, positions: {}'.format(n0['pop_name'], n2['positions']))
print('cell 3: pop_name: {}, positions: {}'.format(n1['pop_name'], n3['positions']))
print('cell 4: pop_name: {}, positions: {}'.format(n0['pop_name'], n4['positions']))



cell 0: pop_name: Exc, positions: [134 139   0]
cell 1: pop_name: Exc, positions: [123   9   0]
cell 2: pop_name: Exc, positions: [ 10 144   0]
cell 3: pop_name: Exc, positions: [109  32   0]
cell 4: pop_name: Exc, positions: [69 28  0]


In [28]:
# Check that LGN positions are being well distributed
lgn_nodes = list(lgn.nodes())
for i in range(10):
    print(format(lgn_nodes[i]['positions']))

[34 78  0]
[30 88  0]
[34 58  0]
[57  5  0]
[23 18  0]
[100  83   0]
[77 46  0]
[77 82  0]
[19 74  0]
[51 23  0]


In [33]:
# Verify distance calculation working as intended
import numpy as np
x1 = np.array([50, 50])
x2 = np.array([55, 55])
print(np.linalg.norm(x1 - x2))

# r = np.linalg.norm(np.array(source['positions']) - np.array(target['positions']))


7.0710678118654755
